## general imports

In [36]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import math
import random
import copy

# MVR studies
Run the minimal violations ranking algorith for the following cases (all 2016 prem league)
* Basic transfer network
* Financial networks


In [2]:
def choose(n, x):
    f = math.factorial
    return f(n) // f(x) // f(n-x)

def calc_V(A: np.array, pi: list) -> int:
    '''
    Calculate the number of violations occur in the current rankings
    
    Inputs:
        A:    (np.array) the adjacency matrix
        pi:   (list) the current rankings. entry 0 is the ranking for node 0 up to len(A)-1
    Ouputs:
        (int) the number of violations found
    '''
    V = 0
    
    for i in range(len(A)):
        for j in range(len(A)):
            
            indicator = 1 if pi[i] < pi[j] else 0
                    
            V += A[i, j] * indicator
            
    return V

def MVR(A: np.array) -> (list, list, int):
    '''
    Run the MVR algorithm until V can no longer be decreased (for n choose 2 steps)
    
    Inputs:
        A:    (np.array) the adjacency matrix. Entries can be weighted 
    Outputs:
        (list, list, int) the final rankings (pi), the number of violations at each step, number of timesteps to get pi
    '''
    
    # first calculate the number of steps for stopping
    stopping_condition = choose(len(A), 2)
    
    # create a random starting rank
    pi = [i+1 for i in range(len(A))]
    random.shuffle(pi)
    
    # keep track of all V
    Vs = []
    
    # keep track of how many times V as no longer gone down
    V_min_kept = 0
    
    # keep track of the V at the last time step
    V = calc_V(A, pi)
    Vs.append(V)
    
    # keep track of time steps
    t = 0
    
    while True:
        
        # pick any two nodes
        node_1 = random.randint(0, len(A)-1)
        node_2 = random.randint(0, len(A)-1)
        
        # if we chose the same node, just go throught the loop again
        if node_1 == node_2:
            continue
            
        # create a proposed pi
        proposed_pi = copy.deepcopy(pi)
        proposed_pi[node_1] = pi[node_2]
        proposed_pi[node_2] = pi[node_1]
        
        # get a proposed V
        V_proposed = calc_V(A, proposed_pi)
        
        # if new V is less than or equal V, swap
        if V_proposed <= V:
            
            # if they are equal, incremement V_min_kept
            if V == V_proposed:
                V_min_kept += 1
                
            else:
                V_min_kept = 0
                
            # set V and pi to the proposed 
            V = V_proposed
            pi = proposed_pi
            
        # otherwise, increment V_min_kept
        else:
            V_min_kept += 1
        
        # keep track of this V
        Vs.append(V)
        
        # increment t
        t += 1
        
        print(f'stopping condition {V_min_kept}/{stopping_condition}', end='\r')
            
        # if V_min_kept == stopping condition, break
        if V_min_kept == stopping_condition:
            break
        
    return (pi, Vs, t)

### our imports

In [3]:
from src.load import load_basic_financial_transfer_networks, load_basic_transfer_networks, prem_season_transfer_summary


## Step 0: rankings
get the season rankings for the season

In [4]:
season = 2016
season_summ = prem_season_transfer_summary(season)

In [28]:
payout_sort = sorted(season_summ, key=lambda x: sum([a.fee for a in x.transfers if a.direction == 'out']), reverse=True)

In [29]:
[(x.name, sum([a.fee for a in x.transfers if a.direction=='out'])) for x in payout_sort]

[('chelsea', 98.00999999999999),
 ('southampton', 89.50999999999999),
 ('liverpool', 76.842),
 ('leicester city', 59.449999999999996),
 ('everton', 54.72),
 ('watford', 52.290000000000006),
 ('tottenham', 47.06999999999999),
 ('crystal palace', 45.18000000000001),
 ('swansea city', 44.370000000000005),
 ('manchester united', 42.53),
 ('west ham united', 36.9),
 ('manchester city', 31.82),
 ('hull city', 26.01),
 ('west bromwich albion', 24.278000000000002),
 ('sunderland', 21.4),
 ('bournemouth', 21.320999999999998),
 ('middlesbrough', 12.149999999999999),
 ('arsenal', 9.32),
 ('stoke city', 3.42),
 ('burnley', 1.08)]

In [51]:
total_in = lambda club: sum([t.fee for t in club.transfers if t.direction == 'out'])
total_out = lambda club: sum([t.fee for t in club.transfers if t.direction == 'in'])

num_in = lambda club: len([t for t in club.transfers if t.direction=='in'])
num_out = lambda club: len([t for t in club.transfers if t.direction == 'out'])

to_table = []

for club in season_summ:
    t_in = total_in(club)
    t_out = total_out(club)
    
    net_in = t_in - t_out
    
    ts_in = num_in(club)
    ts_out = num_out(club)
    
    to_table.append((club.name, club.rank, ts_in, ts_out, t_in, t_out, net_in))
    
df = pd.DataFrame(to_table, columns =['club', 'rank', 'incoming players', 'outgoing players', 'money in', 'money out', 'net in'])
df.head(20)

,club,rank,incoming players,outgoing players,money in,money out,net in
0,chelsea,1,27,29,98.010,119.520,-21.510
1,tottenham,2,7,8,47.070,75.150,-28.080
2,manchester city,3,25,22,31.820,193.500,-161.680
3,liverpool,4,18,21,76.842,71.910,4.932
4,arsenal,5,13,12,9.320,101.736,-92.416
5,manchester united,6,9,15,42.530,166.500,-123.970
6,everton,7,18,18,54.720,90.000,-35.280
7,southampton,8,9,10,89.510,62.010,27.500
8,bournemouth,9,19,27,21.321,36.626,-15.305
9,west bromwich albion,10,12,13,24.278,47.790,-23.512


In [42]:
df.sort_values('net in', ascending=False).head(20)

,club,rank,incoming players,outgoing players,money in,money out,net in
7,southampton,8,9,10,89.510,62.010,27.500
3,liverpool,4,18,21,76.842,71.910,4.932
14,swansea city,15,18,14,44.370,52.290,-7.920
16,watford,17,26,28,52.290,63.415,-11.125
19,sunderland,20,21,21,21.400,36.680,-15.280
8,bournemouth,9,19,27,21.321,36.626,-15.305
17,hull city,18,20,15,26.010,44.910,-18.900
0,chelsea,1,27,29,98.010,119.520,-21.510
11,leicester city,12,16,17,59.450,82.440,-22.990
9,west bromwich albion,10,12,13,24.278,47.790,-23.512


## 1. Basic transfer network

In [5]:
tn = load_basic_transfer_networks()[season]

In [6]:
# ordering 0 to n-1 is given by G.nodes()
tn_adjacency = nx.convert_matrix.to_numpy_matrix(tn.G)

In [7]:
# run mvr
final_rankings, _, _ = MVR(tn_adjacency)

In [12]:
# create a dictionary for the ordering of nodes
node_order = {i+1: club for i, club in enumerate(tn.G.nodes())}


In [13]:
# now go through final rankings to get the ordering of only premier league teams
prem_mvr = [node_order[i] for i in final_rankings if node_order[i] in tn.league_clubs]
prem_mvr

['west bromwich albion',
 'tottenham',
 'liverpool',
 'hull city',
 'burnley',
 'middlesbrough',
 'leicester city',
 'sunderland',
 'bournemouth',
 'crystal palace',
 'west ham united',
 'southampton',
 'chelsea',
 'watford',
 'swansea city',
 'stoke city',
 'arsenal',
 'everton',
 'manchester city',
 'manchester united']

## 2. Financial transfer network


In [17]:
ftn = load_basic_financial_transfer_networks()[season]

In [19]:
ftn_adjacency = nx.convert_matrix.to_numpy_matrix(ftn.G, weight=ftn.edge_key)

In [20]:
final_finance_rankings, _, _ = MVR(ftn_adjacency)

In [21]:
finance_node_order = {i+1: club for i, club in enumerate(ftn.G.nodes())}
prem_finance_mvr = [finance_node_order[i] for i in final_finance_rankings if finance_node_order[i] in ftn.league_clubs]
prem_finance_mvr

['burnley',
 'west ham united',
 'sunderland',
 'manchester city',
 'manchester united',
 'bournemouth',
 'southampton',
 'everton',
 'leicester city',
 'chelsea',
 'liverpool',
 'middlesbrough',
 'swansea city',
 'tottenham',
 'west bromwich albion',
 'crystal palace',
 'watford',
 'hull city',
 'arsenal',
 'stoke city']

# Analysis

In [50]:
pltn_mvr = ['west bromwich albion',
 'tottenham',
 'liverpool',
 'hull city',
 'burnley',
 'middlesbrough',
 'leicester city',
 'sunderland',
 'bournemouth',
 'crystal palace',
 'west ham united',
 'southampton',
 'chelsea',
 'watford',
 'swansea city',
 'stoke city',
 'arsenal',
 'everton',
 'manchester city',
 'manchester united']

# get the number in vs out
unranked = {}
for club in season_summ:
    unranked[club.name] = {'in': num_in(club), 'out': num_out(club), 'rank': club.rank}
    
pltn_mvr_to_table = []
for club in pltn_mvr:
    pltn_mvr_to_table.append((club, unranked[club]['rank'], unranked[club]['in'], unranked[club]['out'], unranked[club]['in'] - unranked[club]['out']))
    
df = pd.DataFrame(pltn_mvr_to_table, columns =['club', 'rank', 'incoming players', 'outgoing players', 'net incoming players'])

df.sort_values('net incoming players', ascending=False)

,club,rank,incoming players,outgoing players,net incoming players
10,west ham united,11,23,15,8
3,hull city,18,20,15,5
14,swansea city,15,18,14,4
18,manchester city,3,25,22,3
4,burnley,16,21,20,1
16,arsenal,5,13,12,1
17,everton,7,18,18,0
7,sunderland,20,21,21,0
15,stoke city,13,13,13,0
9,crystal palace,14,21,21,0


In [56]:
plfn_mvr = ['burnley',
 'west ham united',
 'sunderland',
 'manchester city',
 'manchester united',
 'bournemouth',
 'southampton',
 'everton',
 'leicester city',
 'chelsea',
 'liverpool',
 'middlesbrough',
 'swansea city',
 'tottenham',
 'west bromwich albion',
 'crystal palace',
 'watford',
 'hull city',
 'arsenal',
 'stoke city']

unranked = {}
for club in season_summ:
    unranked[club.name] = {'in': total_in(club), 'out': total_out(club), 'rank': club.rank}
    
plfn_to_table = []
for club in plfn_mvr:
    plfn_to_table.append((club, unranked[club]['rank'], unranked[club]['in'], unranked[club]['out'], unranked[club]['in'] - unranked[club]['out']))
    
df = pd.DataFrame(plfn_to_table, columns =['club', 'rank', 'incoming money', 'outgoing money', 'net spending'])
df.sort_values('net spending').head(20)
    

,club,rank,incoming money,outgoing money,net spending
3,manchester city,3,31.820,193.500,-161.680
4,manchester united,6,42.530,166.500,-123.970
18,arsenal,5,9.320,101.736,-92.416
1,west ham united,11,36.900,96.840,-59.940
15,crystal palace,14,45.180,91.170,-45.990
0,burnley,16,1.080,39.870,-38.790
7,everton,7,54.720,90.000,-35.280
19,stoke city,13,3.420,33.553,-30.133
13,tottenham,2,47.070,75.150,-28.080
11,middlesbrough,19,12.150,36.770,-24.620
